In [ ]:
from pyblock3.algebra.fermion_symmetry import Z2, Z22, U1, U11

In [ ]:
sym1 = Z2(0)
sym2 = Z2(1)
print("Z2: %s + %s = %s"%(sym1, sym2, sym1+sym2))
print("Z2: %s + %s = %s"%(sym2, sym2, sym2+sym2))
print("Z2: %s - %s = %s"%(sym1, sym2, sym1-sym2))
print("\n")
sym1 = Z22(0,1)
sym2 = Z22(1,0)
print("Z2*Z2: %s + %s = %s"%(sym1, sym2, sym1+sym2))
print("Z2*Z2: %s + %s = %s"%(sym2, sym2, sym2+sym2))
print("Z2*Z2: %s - %s = %s"%(sym1, sym2, sym1-sym2))
print("\n")

sym1 = U1(1)
sym2 = U1(2)
print("U1: %s + %s = %s"%(sym1, sym2, sym1+sym2))
print("U1: %s + %s = %s"%(sym2, sym2, sym2+sym2))
print("U1: %s - %s = %s"%(sym1, sym2, sym1-sym2))
print("\n")

sym1 = U11(1,1)
sym2 = U11(2,0)
print("U1*U1: %s + %s = %s"%(sym1, sym2, sym1+sym2))
print("U1*U1: %s + %s = %s"%(sym2, sym2, sym2+sym2))
print("U1*U1: %s - %s = %s"%(sym1, sym2, sym1-sym2))
print("\n")


In [ ]:
from pyblock3.algebra.symmetry import BondInfo
from pyblock3.algebra.fermion import SparseFermionTensor

bond = BondInfo({Z2(0):3, Z2(1):5})
T = SparseFermionTensor.random((bond,)*4, pattern="+--+", dq=Z2(1))


In [ ]:
print("symmetry pattern:%s"%T.pattern)
print("number of blocks=%i"%len(T.blocks))
print("First block",type(T.blocks[0]))

for ix, iblk in enumerate(T.blocks):
    print("block %i: %s - %s - %s + %s = %s"%(ix, *iblk.q_labels, T.dq), "shape:", iblk.shape)

In [ ]:
bond1 = BondInfo({U11(0,0):3, U11(1,1):2, U11(1,-1):4, U11(2,0):5})
bond2 = BondInfo({U11(1,1):2, U11(1,-1):4})

T_u11 = SparseFermionTensor.random((bond1,bond1,bond2,bond2), pattern="+--+", dq=U11(1,1))
print("symmetry pattern:%s"%T.pattern)
print("number of blocks=%i"%len(T_u11.blocks))
print("First block",type(T_u11.blocks[0]))
for ix, iblk in enumerate(T_u11.blocks):
    print("block %i: %s - %s - %s + %s = %s"%(ix, *iblk.q_labels, T.dq), "shape:", iblk.shape)

In [ ]:
import numpy as np

bond = BondInfo({Z2(0):3, Z2(1):5})
T1 = SparseFermionTensor.random((bond,)*3, pattern="++-", dq=Z2(1))


scalar = np.tensordot(T1,T1,axes=((0,1,2),(0,1,2)))
print(scalar)

T2 = SparseFermionTensor.random((bond,)*3, pattern="++-", dq=Z2(1))

out_T1T2 = np.tensordot(T1, T2, axes=((2,),(0,))) # T1[i,j,k] T2[k,l,m]
out_T2T1 = np.tensordot(T2, T1, axes=((0,),(2,))).transpose((2,3,0,1)) # T2[k,l,m] T1[i,j,k]
print((out_T1T2-out_T2T1).norm(), out_T1T2.dq) # \hat{T1}\hat{T2} != \hat{T2}\hat{T1}

# applying Pizorn's Algorithm
new_T2 = T2.copy()
new_T2._local_flip([0])
new_T2._global_flip()

new_out_T2T1 = np.tensordot(new_T2, T1, axes=((0,),(2,))).transpose((2,3,0,1))
print((out_T1T2-new_out_T2T1).norm(), new_out_T2T1.dq)

In [ ]:
bond = BondInfo({Z2(0):1, Z2(1):2})
T = SparseFermionTensor.random((bond,)*2, pattern="+-", dq=Z2(0)) #T[i,j]
Tx = np.transpose(T, (1,0)) #Tx[j,i]
Ty = T.dagger # hermitian conjugate of the operator # Ty[ji]

for iblk1, iblk2, iblk3, in zip(T, Tx, Ty):
    print(np.asarray(iblk1.transpose([1,0])).ravel(), np.asarray(iblk2).ravel(), np.asarray(iblk3).ravel())

In [ ]:
bond = BondInfo({Z2(0):1, Z2(1):2})
T = SparseFermionTensor.random((bond,)*4, pattern="++--", dq=Z2(1)) #T[i,j]
qpn_partition = (Z2(0), Z2(1)) # net symmetry Z2(0) on U, net symmetry Z2(1) on V

for max_bond in range(4, 10):
    U, S, V = T.tensor_svd(left_idx=[0,2], qpn_partition=qpn_partition, absorb=None, max_bond=max_bond)
    tmp = np.tensordot(U, S, axes=((-1,),(0,)))
    new_T = np.tensordot(tmp, V, axes=((-1,),(0,))).transpose([0,2,1,3])
    print("max_bond=%i, err=%.8f"%(max_bond, (new_T-T).norm()))

In [ ]:
bond = BondInfo({Z2(0):1, Z2(1):2})
T = SparseFermionTensor.random((bond,)*4, pattern="++--", dq=Z2(1)) #T[i,j]
qpn_partition = (Z2(0), Z2(1)) # net symmetry Z2(0) on U, net symmetry Z2(1) on V


Q, R = T.tensor_qr(left_idx=[0,2], mod='qr')
new_T = np.tensordot(Q, R, axes=((-1,),(0,))).transpose([0,2,1,3])
print("err=%.8f"%((new_T-T).norm()))

L, Q = T.tensor_qr(left_idx=[0,2], mod='lq')
new_T = np.tensordot(L, Q, axes=((-1,),(0,))).transpose([0,2,1,3])
print("err=%.8f"%((new_T-T).norm()))


In [ ]:
from pyblock3.algebra.core import SubTensor

even_even_block = np.zeros((2,2))
even_even_block[1,1] = 2
# q_labels (output symmetry label, input symmetry label)
block1 = SubTensor(reduced=even_even_block, q_labels=(Z2(0), Z2(0))) 

odd_odd_block = np.eye(2)
block2 = SubTensor(reduced=odd_odd_block, q_labels=(Z2(1), Z2(1)))

PN = SparseFermionTensor(blocks=[block1, block2], pattern="+-")

even_even_block = np.zeros((2,2))
block1 = SubTensor(reduced=even_even_block, q_labels=(Z2(0), Z2(0)))

odd_odd_block = np.eye(2)*.5
odd_odd_block[1,1] *= - 1
block2 = SubTensor(reduced=odd_odd_block, q_labels=(Z2(1), Z2(1)))
SZ = SparseFermionTensor(blocks=[block1, block2], pattern="+-")

odd_odd_block = np.zeros((2,2))
odd_odd_block[0,1] = 1  #  |+,->
odd_odd_block[1,0] = -1 # -|-,+>
block = SubTensor(reduced=odd_odd_block, q_labels=((Z2(1), Z2(1))))
singlet = SparseFermionTensor(blocks=[block], pattern="++") # creation operators for both indices


# <psi| N_0| psi>, particle number on 1st site

bra = singlet.dagger
norm = np.tensordot(bra, singlet, axes=((0,1),(1,0))) # bra[j,i] ket[i,j]
op_and_ket = np.tensordot(PN, singlet, axes=((-1,),(0,))) # PN[i,i`] ket[i`,j] -> new_ket[i,j]
expec = np.tensordot(bra, op_and_ket, axes=((0,1),(1,0))) # bra[j,i] new_ket[i,j]
print("<psi|psi>=%.2f, <psi|N_0|psi>=%.2f"%(norm, expec))

# <psi| SZ_0| psi>, particle number on 1st site

op_and_ket = np.tensordot(SZ, singlet, axes=((-1,),(0,))) # bra[j,i] ket[i,j]
expec = np.tensordot(bra, op_and_ket, axes=((0,1),(1,0))) # PN[i,i`] ket[i`,j] -> new_ket[i,j]
print("<psi|psi>=%.2f, <psi|SZ_0|psi>=%.2f"%(norm, expec)) # bra[j,i] new_ket[i,j]


In [ ]:
block_1010 = np.zeros((2,2,2,2))
block_1010[1,0,1,0] = block_1010[0,0,0,0] = 1
block_1010[1,1,1,1] = block_1010[0,1,0,1] =-1
block1 = SubTensor(reduced=block_1010, q_labels=(Z2(1),Z2(0),Z2(1),Z2(0)))

block_0011 = np.zeros((2,2,2,2))
block_0011[1,0,1,0] = block_0011[0,1,1,0] = 1
block_0011[1,0,0,1] = block_0011[0,1,0,1] =-1
block2 = SubTensor(reduced=block_0011, q_labels=(Z2(0),Z2(0),Z2(1),Z2(1)))

block_0101 = np.zeros((2,2,2,2))
block_0101[0,1,0,1] = block_0101[0,0,0,0] = 1
block_0101[1,1,1,1] = block_0101[1,0,1,0] =-1
block3 = SubTensor(reduced=block_0101, q_labels=(Z2(0),Z2(1),Z2(0),Z2(1)))

block_1100 = np.zeros((2,2,2,2))
block_1100[0,1,1,0] = block_1100[0,1,0,1] = 1
block_1100[1,0,1,0] = block_1100[1,0,0,1] =-1
block4 = SubTensor(reduced=block_1100, q_labels=(Z2(1),Z2(1),Z2(0),Z2(0)))

O = SparseFermionTensor(blocks=[block1,block2,block3,block4], pattern="++--")
Ox = O.dagger

print("checking Hermicity",(O-Ox).norm())


In [ ]:
# Note in the case above, the O tensor we generated are labelled by [i,j,j',i']. 
# In quimb interface, the default assumes the operator gate are labelled by [i,j,i',j']
O_to_quimb = O.transpose([0,1,3,2])

from pyblock3.algebra.fermion_ops import H1, Hubbard

Otmp = H1(symmetry=Z2, flat=False)
print((Otmp-O_to_quimb).norm())



In [ ]:
bond = BondInfo({Z2(0):3, Z2(1):5})
T1 = SparseFermionTensor.random((bond,)*3, pattern="++-", dq=Z2(1))
Tf1 = T1.to_flat()
print(T1.__class__, Tf1.__class__)

##########################
####Tensor Contraction####
##########################
scalar = np.tensordot(T1,T1,axes=((0,1,2),(0,1,2)))
scalar_f = np.tensordot(Tf1,Tf1,axes=((0,1,2),(0,1,2)))
print(scalar, scalar_f)

T2 = SparseFermionTensor.random((bond,)*3, pattern="++-", dq=Z2(1))
Tf2 = T2.to_flat()

out_T1T2 = np.tensordot(T1, T2, axes=((2,),(0,)))
outf_T1T2 = np.tensordot(Tf1, Tf2, axes=((2,),(0,)))

# convert them to the same data class before operations
print((out_T1T2.to_flat()-outf_T1T2).norm(), (outf_T1T2.to_sparse()-out_T1T2).norm())

##########################
########Tensor SVD########
##########################

bond = BondInfo({Z2(0):1, Z2(1):2})
T = SparseFermionTensor.random((bond,)*4, pattern="++--", dq=Z2(1)) #T[i,j]
Tf = T.to_flat()
qpn_partition = (Z2(0), Z2(1)) # net symmetry Z2(0) on U, net symmetry Z2(1) on V

for max_bond in range(4, 10):
    U, S, V = Tf.tensor_svd(left_idx=[0,2], qpn_partition=qpn_partition, absorb=None, max_bond=max_bond)
    tmp = np.tensordot(U, S, axes=((-1,),(0,)))
    new_Tf = np.tensordot(tmp, V, axes=((-1,),(0,))).transpose([0,2,1,3])
    print("max_bond=%i, SVD err=%.8f"%(max_bond, (new_Tf-Tf).norm()))

##########################
########Tensor QR#########
##########################

bond = BondInfo({Z2(0):1, Z2(1):2})
T = SparseFermionTensor.random((bond,)*4, pattern="++--", dq=Z2(1)) #T[i,j]
Tf = T.to_flat()
qpn_partition = (Z2(0), Z2(1)) # net symmetry Z2(0) on U, net symmetry Z2(1) on V


Q, R = Tf.tensor_qr(left_idx=[0,2], mod='qr')
new_Tf = np.tensordot(Q, R, axes=((-1,),(0,))).transpose([0,2,1,3])
print("QR err=%.8f"%((new_Tf-Tf).norm()))

L, Q = Tf.tensor_qr(left_idx=[0,2], mod='lq')
new_Tf = np.tensordot(L, Q, axes=((-1,),(0,))).transpose([0,2,1,3])
print("LQ err=%.8f"%((new_Tf-Tf).norm()))

